In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [174]:
# 브라우저 실행 -> 네이버 지도 이동
driver = wb.Chrome()
driver.get("https://map.naver.com/p?c=15.00,0,0,0,dh")
driver.maximize_window()

In [176]:
# 검색창에 동명동맛집 입력 -> 검색
# 선택자가 랜덤으로 변하는 경우가 있다.
#  How? 변하지 않는 선택자를찾자 ex) class, 계층선택자
driver.find_element(By.CSS_SELECTOR,"input.input_search").send_keys("동명동맛집",Keys.ENTER)

- id(#input_search1736919279757)의 난수값이 계속 변경되므로 이때는 class(.input_search)를 사용하자!

In [73]:
# 검색된 가게들의 태그를 수집
# Why? -> 하나씩 클릭
# 가게이름 첫 번째꺼 클릭 -> 오른쪽 데이터 수집 -> 두 번째.....
title = driver.find_elements(By.CSS_SELECTOR,"#_pcmap_list_scroll_container > ul > li > div.CHC5F > a.tzwk0")

In [75]:
len(title)

0

## 화면에는 보이는데.. 접근이 안된다..?
- iframe을 의심해봐라
- iframe : 다른 서버에 저장된 데이터를 보여주기만 하는 태그
- 우리가 필요한건 실제 서버의 주소 -> src속 경로를 파악
- 상대경로이면 -> 절대경로로 변경 -> req로 크롤링할 때 사용하는 방식
- But! selenium은 브라우저 자체를 제어하는 목적 -> 링크를 옮겨가는 게 아닌 화면에서 조작 실행
- How? driver가 현재 가지고 있는 화면(처음 검색된 결과)을 iframe으로 변경

In [178]:
# 원본데이터에서 iframe으로 변경
# 이동할 iframe태그 찾기 -> 드라이버에게 변경 명령 내기기
# 원본에서 iframe으로 변경 -> switch_to.frame(iframe의 태그위치(선택자))
search_iframe = driver.find_element(By.CSS_SELECTOR,"#searchIframe")
driver.switch_to.frame(search_iframe)

In [80]:
# 다시 수집해보기
title = driver.find_elements(By.CSS_SELECTOR,"#_pcmap_list_scroll_container > ul > li > div.CHC5F > a.tzwk0")
len(title)

10

### 전체 결과가 아니다 -> 스크롤링 필요
- 기존방식 : body태그에게 키보드의 end를 보내는 방식을 사용
- 간혹 iframe인 경우에는 body에게 스크롤을 명령할 수 없다.
#### 해결책 : iframe속 데이터를 담고 있는 태그(div) -> 옆에 scroll이라고 태그가 붙은 div가 존재

In [180]:
div = driver.find_element(By.CSS_SELECTOR,"#_pcmap_list_scroll_container")
div.send_keys(Keys.END)

- 기존 스크롤 : 임의의 횟수 만큼 for문 작성
- 단점 : 반복횟수가 낭비, 부족할수도 있다
#### while문을 통해서 알아서 끝까지 반복하는 문법
- 대화 : 스크롤을 내리기 전 데이터와 스크롤을 내린 후 데이터를 비교해라!
- 만약에 두 데이터가 다르다면? 스크롤이 된다 -> 스크롤 해라
- 만약에 두 데이터가 같다면? 스크롤이 안된다 -> 스크롤을 멈춰라

In [182]:
while True:
    # 스크롤 내리기 전 데이터 저장
    first_data = driver.page_source
    # 스크롤 내리기
    div.send_keys(Keys.END)
    # 시간부여
    time.sleep(0.5)
    # 내린 후 데이터 저장
    last_data = driver.page_source
    # 멈추는 조건 부여
    if first_data == last_data:
        break

In [ ]:
# 다른 코드에 활용할 때는 div -> body태그로 변경
body = drive.find_element(By.CSS_SELECTOR,"body")
while True:
    # 스크롤 내리기 전 데이터 저장
    first_data = driver.page_source
    # 스크롤 내리기
    body.send_keys(Keys.END)
    # 시간부여
    time.sleep(0.5)
    # 내린 후 데이터 저장
    last_data = driver.page_source
    # 멈추는 조건 부여
    if first_data == last_data:
        break

In [184]:
# 스크롤 후 데이터 다시 수집
title = driver.find_elements(By.CSS_SELECTOR,"#_pcmap_list_scroll_container > ul > li > div.CHC5F > a.tzwk0")
len(title)

56

In [186]:
# 첫번째 가게 클릭 -> 디테일 정보 얻기 위해서
title[0].click()

In [103]:
# 우측 디테일 창에서 가게이름 수집
title[0].find_elements(By.CSS_SELECTOR,"#_title > div > span.GHAhO")

[]

- 또 iframe 문제!

In [110]:
# 새로운 iframe으로 변환
entry_iframe = driver.find_element(By.CSS_SELECTOR,"#entryIframe")
driver.switch_to.frame(entry_iframe)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#entryIframe"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6E1C580D5+2992373]
	(No symbol) [0x00007FF6E18EBFD0]
	(No symbol) [0x00007FF6E178590A]
	(No symbol) [0x00007FF6E17D926E]
	(No symbol) [0x00007FF6E17D955C]
	(No symbol) [0x00007FF6E18227D7]
	(No symbol) [0x00007FF6E17FF3AF]
	(No symbol) [0x00007FF6E181F584]
	(No symbol) [0x00007FF6E17FF113]
	(No symbol) [0x00007FF6E17CA918]
	(No symbol) [0x00007FF6E17CBA81]
	GetHandleVerifier [0x00007FF6E1CB6A2D+3379789]
	GetHandleVerifier [0x00007FF6E1CCC32D+3468109]
	GetHandleVerifier [0x00007FF6E1CC0043+3418211]
	GetHandleVerifier [0x00007FF6E1A4C78B+847787]
	(No symbol) [0x00007FF6E18F757F]
	(No symbol) [0x00007FF6E18F2FC4]
	(No symbol) [0x00007FF6E18

### 해당 코드가 오류가 나는 이유
- 2번 iframe과 3번 iframe은 서로 모르는 관계(불포함 관계)
- How? 2번 -> 원본 -> 3번

In [188]:
# iframe에서 원본으로 이동하는 방법
driver.switch_to.default_content()

In [190]:
# 다시 iframe으로 이동
entry_iframe = driver.find_element(By.CSS_SELECTOR,"#entryIframe")
driver.switch_to.frame(entry_iframe)

In [192]:
# 우측 디테일 속의 가게 이름 수집
store_name = driver.find_element(By.CSS_SELECTOR,"#_title > div > span.GHAhO")
store_name.text

'쭈꾸미잡스'

### 목적 : 2번째 검색 결과를 클릭 후, 디테일 정보 중 가게 타이틀 수집
- 키포인트 : 현재 위치는 3번 iframe
- 1) 원본으로 이동
  2) 2번 iframe으로 이동
  3) 가게 클릭
  4) 원본으로 이동
  5) 3번 iframe 이동
  6) 가게 타이틀 수집

In [204]:
driver.switch_to.default_content()
driver.switch_to.frame(search_iframe)
title[1].click()
time.sleep(1)
driver.switch_to.default_content()
entry_iframe = driver.find_element(By.CSS_SELECTOR,"#entryIframe") # 화면 초기화(화면 깜박거림)가 되기 때문에 다시 찾아와야 함!
driver.switch_to.frame(entry_iframe)
store_name = driver.find_element(By.CSS_SELECTOR,"#_title > div > span.GHAhO")
store_name.text

'아이다요'

In [206]:
# 반복문을 통해서 모든 가게 정보 수집
name_list = []
for i in range(len(title)):
    driver.switch_to.default_content()
    driver.switch_to.frame(search_iframe)
    title[i].click()
    time.sleep(3)
    driver.switch_to.default_content()
    entry_iframe = driver.find_element(By.CSS_SELECTOR,"#entryIframe") # 화면 초기화(화면 깜박거림)가 되기 때문에 다시 찾아와야 함!
    driver.switch_to.frame(entry_iframe)
    store_name = driver.find_element(By.CSS_SELECTOR,"#_title > div > span.GHAhO")
    name_list.append(store_name.text)

In [207]:
name_list

['쭈꾸미잡스',
 '아이다요',
 '캬베츠',
 '왕산식당 동명점',
 '아이다요',
 '아비오',
 '비미',
 '수을상',
 '라그로쏘',
 '컴피코지',
 '동백카츠 동명점',
 '대풍쪽갈비 본점',
 '황톳길 본점',
 '육값어치',
 '코다',
 '한식경',
 '녹지',
 '오감탄 본점',
 '온우',
 '천지라멘',
 '파블로츄러스',
 '소보쿠',
 '동명동2F',
 '소바 쿄다이',
 '규슈 동명본점',
 '틸트틸트 동명',
 '영원',
 '올데이우동 동명점',
 '윤슬베이커리',
 '폴리아',
 '부엌간 동명 직영점',
 '900달러',
 '꾀꾀로',
 '그랩앤고 크레페 동명본점',
 '구미구미',
 '라이드올스테이카페클럽',
 '연어쁘다 광주본점',
 '블루웨일',
 '동명식빵',
 '사보타지',
 '녹는당',
 '차오르면',
 '돌애',
 '사월의파랑',
 '달곰식당',
 '퀴비',
 '오월의동명동',
 '밀물',
 '포카포카',
 '선데이즈가든',
 '송경솥밥',
 '오펜스커피',
 '카프테일러',
 '클로우치',
 '중정',
 '동명 모락']

In [211]:
len(name_list)

56

## 오늘 파일 정리
- 아이디가 바뀌는 선택자 존재 -> class, 계층을 활용하자
- selenium에서 iframe을 처리하는 방법
- 1) 원본데이터 -> iframe 이동 : driver.switch_to.frame(이동할 iframe 위치)
  2) frame -> frame 불가능 = frame -> 원본 -> frame 이동
     - driver.switch_to.default_content()로 이동 후에 변환 가능
- 태그를 수집할 때 그냥 검색된거 수집하지 말고 태그를 분석하자
  - 글자를 클릭했을 때 화면이 바뀐다면 반드시 a태그를 수집
  - 글자를 입력하는 공간을 수집할 때는 반드시 input태그를 수집
  - 광고 같은 키워드로 인해서 클릭이 되지 않는 경우 -> 넓은 영역을 수집
- iframe에서 스크롤 작업할 때 body가 안되는 경우가 있다
  - 영역을 포함한 div들 중 옆에 태그로 scroll을 확인 후, 그 div에게 end 보내기
- 앞으로 스크롤 내리는 작업할 때는 while문 코드 사용하기

In [ ]:
from selenium import webdriver as wb
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
 
 
# 사이트 실행 및 요청
driver = wb.Chrome()
driver.maximize_window()
driver.get('https://map.naver.com/p?c=15.00,0,0,0,dh1')
 
input = driver.find_element(By.CSS_SELECTOR,'.input_search')
 
input.send_keys('동명동 맛집')
input.send_keys(Keys.ENTER)
time.sleep(3)
 
search_iframe = driver.find_element(By.CSS_SELECTOR, '#searchIframe')
iframe_src = search_iframe.get_attribute('src')
 
driver.get(iframe_src)
time.sleep(2)
title_list=[]
container = driver.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container')
title = driver.find_elements(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li > div > a > div > div > span.TYaxT')
scroll_count = 0  
while scroll_count < 5:
    container = driver.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container')
    for _ in range(10):
        container.send_keys(Keys.END)
        time.sleep(0.2)
 
    titles = driver.find_elements(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li > div > a > div > div > span.TYaxT')
    for title in titles:
        text = title.text.strip()
        if text and text not in title_list:
            title_list.append(text)
 
    next_buttons = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(7) > svg')
    if next_buttons:
        next_buttons.click()  
        scroll_count += 1  
        time.sleep(2) 

 
# 드라이버 종료
driver.quit()
len(title_list)